AI多模态大模型发展至今，每年都有非常优秀的工作产出，按照当前模型设计思路，多模态大模型的架构主要包括以下几个部分：

- 模态编码器(`Modality Encoder, ME`)：负责将不同模态的输入编码成特征。常见的编码器包括图像的NFNet-F6、ViT、CLIP ViT等，音频的Whisper、CLAP等，视频编码器等。

- 输入投影器(`Input Projector`)：负责将其他模态的特征投影到文本特征空间，并与文本特征一起输入给语言模型。常用的投影器包括线性投影器、MLP、交叉注意力，Q-Former，P-Former等。

- 语言模型骨架(`LLM Backbone`)：利用预训练的语言模型，负责处理各种模态的特征，进行语义理解、推理和决策。常用的语言模型包括Flan-T5、ChatGLM、UL2等。

- 输出投影器(`Output Projector`)：负责将语言模型输出的信号转换成其他模态的特征，以供后续模态生成器使用。常用的投影器包括Tiny Transformer、MLP等。

- 模态生成器(`Modality Generator, MG`)：负责生成其他模态的输出。常用的生成器包括图像的Stable Diffusion、视频的Zeroscope、音频的AudioLDM等。

本文一手会详细解读AI多模态架构中的输入投影器(Input Projector)，并从线性投影器（`Linear Projector`）、多层感知器（`Multi-Layer Perception, MLP`）和交叉注意力（`Cross-Attention`）三个角度，总结当前主流的工作方案！

多模态大模型需要处理不同类型的输入数据，如图像、文本、音频等。为了将这些不同的数据转换到一个共同的表示空间，引入了输入投影器。

## Linear Projector（线性投影器, LP）
线性投影器是一种简单的投影方法，通过线性变换将输入数据映射到目标表示空间。

特点:
- 简单高效：计算速度快，易于实现。
- 参数少：所需参数较少，适合参数敏感的场景。

优缺点:
- 优点：高效，适合大规模数据处理。
- 缺点：表达能力有限，无法捕捉复杂的非线性关系。

In [1]:
import torch
import torch.nn as nn

class LinearProjector(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearProjector, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

# 示例输入
image_features = torch.randn(32, 2048)  # 32个样本，每个样本2048维
text_features = torch.randn(32, 300)    # 32个样本，每个样本300维

# 投影到相同的表示空间
projector = LinearProjector(2048, 512)
projected_image_features = projector(image_features)

projector = LinearProjector(300, 512)
projected_text_features = projector(text_features)
print(projected_text_features)

tensor([[-0.3049,  0.0563, -0.2754,  ..., -0.6008,  0.4132, -0.5658],
        [ 0.4569,  0.8294,  0.0804,  ..., -1.2296,  0.2409,  0.1005],
        [-0.8232, -0.4658,  0.0242,  ..., -0.2850, -0.8126,  0.7600],
        ...,
        [-0.6319, -0.6294,  0.7677,  ..., -0.5352,  0.0355,  0.7557],
        [ 0.7881,  0.2220,  0.0029,  ...,  0.6534,  1.0186, -0.3021],
        [-0.6378,  0.3613,  0.7785,  ..., -0.3830, -0.7432,  1.1319]],
       grad_fn=<AddmmBackward0>)


## Multi-Layer Perception（多层感知器, MLP）

多层感知器是一种神经网络，由多层线性变换和非线性激活函数组成，能够捕捉输入数据的复杂非线性关系。

特点：
- 非线性：能够表示和捕捉复杂的非线性关系。
- 层次结构：通过多层结构逐步提取特征，表示数据更好。

优缺点:
- 优点：强大的表示能力，能够捕捉复杂的特征和模式。
- 缺点：计算复杂度高，训练时间长，容易过拟合。

In [2]:
import torch
import torch.nn as nn

class MLPProjector(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLPProjector, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 示例输入
image_features = torch.randn(32, 2048)
text_features = torch.randn(32, 300)

# 投影到相同的表示空间
mlp_projector = MLPProjector(2048, 1024, 512)
projected_image_features = mlp_projector(image_features)

mlp_projector = MLPProjector(300, 512, 512)
projected_text_features = mlp_projector(text_features)
print(projected_text_features)

tensor([[ 0.3541,  0.0064,  0.0598,  ..., -0.1948, -0.1849,  0.2483],
        [ 0.0916, -0.0562, -0.0709,  ..., -0.3561,  0.0189, -0.5898],
        [ 0.3568,  0.1058, -0.3574,  ..., -0.3725, -0.1003, -0.2642],
        ...,
        [ 0.2145,  0.0248, -0.1390,  ...,  0.0804, -0.3632, -0.2873],
        [ 0.3753, -0.0794, -0.1604,  ..., -0.0943,  0.0925, -0.2909],
        [ 0.0123, -0.0384,  0.0872,  ..., -0.0048, -0.1921, -0.3131]],
       grad_fn=<AddmmBackward0>)


## Cross-Attention（交叉注意力）
交叉注意力机制在多模态模型中非常重要，通过计算不同模态间的注意力权重，实现信息的交互和融合。

特点：
- 信息融合：在不同模态间有效地交换和融合信息。
- 权重自适应：根据输入动态计算注意力权重，更加灵活和智能。

优缺点：
- 优点：在不同模态之间高效地捕捉相关性，适应不同类型的输入。
- 缺点：计算复杂度较高，尤其在处理长序列输入时。

In [4]:
import torch
import torch.nn as nn

class CrossAttention(nn.Module):
    def __init__(self, dim, num_heads):
        super(CrossAttention, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads)

    def forward(self, query, key, value):
        attn_output, _ = self.multihead_attn(query, key, value)
        return attn_output

# 示例输入
# 10个图像特征序列，每个特征序列32个时间步，每个时间步512维
image_features = torch.randn(10, 32, 512)
# 20个文本特征序列，每个特征序列32个时间步，每个时间步512维
text_features = torch.randn(20, 32, 512)

# 使用交叉注意力
cross_attention = CrossAttention(dim=512, num_heads=8)
# 让图像特征作为query，文本特征作为key和value
projected_features = cross_attention(image_features, text_features, text_features)
print(projected_features)

tensor([[[-9.5709e-02, -5.6605e-02, -2.4755e-02,  ..., -4.4275e-02,
          -3.2871e-02, -1.2942e-01],
         [ 2.8259e-01, -2.2002e-01, -5.4394e-02,  ...,  1.9745e-02,
          -1.4952e-01,  5.5191e-02],
         [ 1.1994e-01, -3.6203e-02,  1.8334e-04,  ...,  9.5060e-02,
          -6.7025e-02, -9.0659e-02],
         ...,
         [-7.2133e-02,  1.8860e-02, -1.3087e-01,  ...,  5.9735e-02,
          -1.6699e-01,  5.0280e-02],
         [-4.0124e-02,  5.9835e-02,  1.3879e-01,  ..., -1.2142e-01,
          -1.2244e-01,  1.8140e-01],
         [ 9.6986e-03,  1.4695e-01,  4.4646e-03,  ...,  1.5083e-02,
          -2.2865e-02, -2.5747e-02]],

        [[-5.1258e-02, -1.2547e-01,  6.6216e-02,  ...,  5.8145e-02,
          -3.2417e-02, -9.4656e-02],
         [ 2.9587e-01, -2.0168e-01,  1.5938e-02,  ..., -1.6048e-03,
          -4.9942e-02,  9.3270e-02],
         [-3.9374e-02, -1.4750e-02,  4.6779e-03,  ...,  9.9899e-02,
          -4.0618e-03, -1.7834e-01],
         ...,
         [-6.0275e-02, -5

## 总结

1. 线性投影器简单高效，适用于计算资源有限的场景；
2. 多层感知器具有强大的表示能力，适用于需要捕捉复杂关系的任务；
3. 交叉注意力在多模态信息融合中表现出色，尤其适用于需要跨模态交互的任务。